# Numerical Initializers Development
Zakariya Abugrin - 04.09.2023

## Introductions

Numerical solutions consist of two steps as following:
1. Initialization: in this step a system of linear equations is created based on `Model` class using numerical initializers (`NumericalInitializer`).
2. Solving the system of linear equations using direct solvers, iterative solvers, or neurical solvers. 
3. Steps 1 and 2 are repeated until a final timestep is reached.

In this notebook, `NumericalInitializer` class is developed for `FDM`.

In [1]:
import reservoirflow as rf

print(rf.__version__)

0.1.0


In [2]:
grid = rf.grids.Cartesian(
    nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
)

fluid = rf.fluids.SinglePhase(mu=0.5, B=1, dtype="double")

model = rf.models.BlackOil(grid, fluid, dtype="double", pi=4000, verbose=True)
model.set_well(id=4, q=-600, s=1.5, r=3.5)
model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
# model.compile(stype="numerical", method="fdm")
# model.compile(stype="analytical", method="-")
# model.compile(stype="neurical", method="pinn")

[info] the model was initialized.
[info] model compressibility (comp) was calculated.
[info] a well in cell 4 was set.
[info] boundary in cell 0 was set to constant pressure.
[info] boundary in cell 5 was set to constant rate.


In [ ]:
model.get_cells_eq()

{1: (defaultdict(int, {p1: 85.2012000000000, p2: -28.4004000000000}),
  227203.200000000),
 2: (defaultdict(int,
              {p1: 28.4004000000000,
               p3: 28.4004000000000,
               p2: -56.8008000000000}),
  0),
 3: (defaultdict(int,
              {p2: 28.4004000000000,
               p4: 28.4004000000000,
               p3: -56.8008000000000}),
  0),
 4: (defaultdict(int, {p3: 28.4004000000000, p4: -28.4004000000000}),
  600.000000000000)}

In [ ]:
A = model.init_A(False)
d = model.init_d(False)
print(f"A:{A}")
print(f"d:{d}")

A:[[-28.4004  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]]
d:None


In [ ]:
model.get_matrices_vectorized(False, True)

(array([[-85.2012,  28.4004,   0.    ,   0.    ],
        [ 28.4004, -56.8008,  28.4004,   0.    ],
        [  0.    ,  28.4004, -56.8008,  28.4004],
        [  0.    ,   0.    ,  28.4004, -28.4004]]),
 array([[-227203.2],
        [      0. ],
        [      0. ],
        [    600. ]]))

In [ ]:
model.verbose = False
model.init_matrices(True, True)

(<4x4 sparse matrix of type '<class 'numpy.float64'>'
 	with 10 stored elements in List of Lists format>,
 <4x1 sparse matrix of type '<class 'numpy.float64'>'
 	with 2 stored elements in List of Lists format>)

In [ ]:
model.run(10, print_arrays=False)

[info] Simulation run started: 10 timesteps.


[step]: 10 - : 100%|██████████| 10/10 [00:00<00:00, 333.32steps/s]

[info] Simulation run of 10 steps finished in 0.04 seconds.
[info] Material Balance Error: -9.387122190673836e-10.
